<a href="https://colab.research.google.com/github/pepemesquita/CNC/blob/main/Relat%C3%B3rio_2_Pedro_Henrique_Lima_de_Mesquita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Relatório 2 - Sistemas de Equações Lineares**
Aluno: Pedro Henrique Lima de Mesquita

Matricula: 20200862


#Imports

In [ ]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
import time
import psutil
import math

#Métodos

##Eliminação-Gauss

In [ ]:
def gauss(n, A, b):
  det = 1
  info = 0

  for j in range(n - 1):
      p = j
      Amax = abs(A[j][j])

      for k in range(j + 1, n):
          if abs(A[k][j]) > Amax:
              Amax = abs(A[k][j])
              p = k

      if p != j:
          for k in range(n):
              t = A[j][k]
              A[j][k] = A[p][k]
              A[p][k] = t

          det = det * A[j][j]

      if abs(A[j][j]) != 0:
          r = 1 / A[j][j]

          for i in range(j + 1, n):
              mult = A[i][j] * r
              A[i][j] = 0

              for k in range(j + 1, n):
                A[i][k] = A[i][k] - mult * A[j][k]

              b[i] = b[i] - mult * b[j]

  if info == 0 and abs(A[n-1][n-1]) == 0:
      info = n

  return A, b, det, info



# Exemplo de uso
n = 3
A = [[1, -1, 6], [-3, -1, 2], [ -2, 1, 2]]
b = [8, -11, -3]

result = gauss(n, A, b)

if result:
    matriz_gauss = np.matrix(result[0])
    vetor = result[1]
    determinante = result[2]
    info = result[3]

    print(f"Matriz Gauss:\n{matriz_gauss}\n")

    print(f"Vetor transformado b: {vetor}\n")

    print(f"Determinante: {determinante}\n")

    print(f"Info: {info}")


Matriz Gauss:
[[-3.         -1.          2.        ]
 [ 0.          1.66666667  0.66666667]
 [ 0.          0.          7.2       ]]

Vetor transformado b: [8, -8.333333333333334, -15.0]

Determinante: -5.0

Info: 0


##Decomposição-LU

In [ ]:
def decomposicao(m, n, A):
    pivo = list(range(m))  # Corrigido: inicializar a lista de pivôs
    pdu = 1
    info = 0

    for j in range(min(m, n)):  # Corrigido: limite superior do loop
        p = j
        Amax = abs(A[j][j])

        for k in range(j + 1, m):
            if abs(A[k][j]) > Amax:
                Amax = abs(A[k][j])
                p = k

        if p != j:
            for k in range(n):  # Corrigido: ajustar o loop para incluir todas as colunas
                t = A[j][k]
                A[j][k] = A[p][k]
                A[p][k] = t
            i = pivo[j]
            pivo[j] = pivo[p]
            pivo[p] = i
            pdu = -pdu

        pdu = pdu * A[j][j]

        if abs(A[j][j]) != 0:
            r = 1 / A[j][j]

            for i in range(j + 1, m):  # Corrigido: limite superior do loop
                mult = A[i][j] * r
                A[i][j] = mult

                for k in range(j + 1, n):
                    A[i][k] = A[i][k] - mult * A[j][k]

        else:
            if info == 0:
                info = j

    return A, pivo, pdu, info

# Exemplo de uso:
m = 3
n = 3
A = [[1, -1, 6], [-3, -1, 2], [ -2, 1, 2]]
result = decomposicao(m, n, A)

if result:
    matriz_decomposicao = np.matrix(result[0])
    pivo = (result[1])
    pdu = (result[2])
    info = (result[3])

    print(f"Matriz de Decomposição:\n{matriz_decomposicao}\n")
    print(f"Pivo: {pivo}\n")
    print(f"PDU: {pdu}\n")
    print(f"Info: {info}")

Matriz de Decomposição:
[[-3.         -1.          2.        ]
 [ 0.66666667  1.66666667  0.66666667]
 [-0.33333333 -0.8         7.2       ]]

Pivo: [1, 2, 0]

PDU: -36.0

Info: 0


##Cholesky

In [ ]:
import math

def cholesky(n, A):
    det = 1
    info = 0

    for j in range(n):
        soma = 0

        for k in range(j):
            soma += A[j][k] * A[j][k]

        t = A[j][j] - soma

        if t > 0:
            A[j][j] = math.sqrt(t)
            r = 1 / A[j][j]
            det *= t
        else:
            info = j
            print("Matriz não é definida positiva")
            return None  # Retorna None em caso de falha

        for i in range(j + 1, n):
            soma = 0

            for k in range(j):
                soma += A[i][k] * A[j][k]

            A[i][j] = (A[i][j] - soma) * r

    return A, det, info


n = 3
A = [[4, 12, -16], [12, 37, -43], [-16, -43, 98]]

result = cholesky(n, A)

if result:
    matriz_cholesky = np.matrix(result[0])
    determinante = result[1]
    info = result[2]

    print(f"Matriz de Cholesky:\n{matriz_cholesky}\n")
    print(f"Determinante: {determinante}\n")
    print(f"Info: {info}")

Matriz de Cholesky:
[[  2.  12. -16.]
 [  6.   1. -43.]
 [ -8.   5.   3.]]

Determinante: 36.0

Info: 0


#Questão 1

#Questão 2


#Questão 3

#Questão 4

#Questão 5